In [25]:
import pandas as pd
import numpy as np
import string
import re
import pickle

from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.initializers import Constant
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import LearningRateScheduler

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "cp1252")
df

,Emotion,ID,Date,Query,Author,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
# removing hashtags and mentions
def remove_hashatags_and_mentions(text):
    return re.sub(r'#\w*', r' ', re.sub(r'@\w*', r' ', str(text)))

df.Text = df.Text.apply(remove_hashatags_and_mentions)
df

,Emotion,ID,Date,Query,Author,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"http://twitpic.com/2y1zl - Awww, that's a bu..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball. Managed to ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"no, it's not behaving at all. i'm mad. why a..."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
# removing punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df.Text = df.Text.apply(remove_punctuation)
df

,Emotion,ID,Date,Query,Author,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,httptwitpiccom2y1zl Awww thats a bummer Yo...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his Facebook by t...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,I dived many times for the ball Managed to s...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i h...
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up Having no school is the best feel...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDBcom Very cool to hear old Walt intervie...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover Ask me fo...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time Tup...


In [5]:
# prepare data for training

untokenized_texts = np.array(df.Text)
emotions = np.array(df.Emotion)

In [6]:
english_embedding_dict = {}
with open("glove.twitter.27B.200d.txt", "r", encoding="utf-8") as english_glove:
    for line in tqdm(english_glove):
        values = line.split()
        word = remove_punctuation(values[0])
        vectors = np.asarray(values[1:], 'float32')
        english_embedding_dict[word] = vectors

1193514it [01:12, 16461.13it/s]


In [7]:
EMBEDDING_LEN = 200

In [8]:
word_cnt = dict()
for text in tqdm(untokenized_texts):
    for word in text.split():
        word_cnt[word] = word_cnt.get(word, 0) + 1
clean_texts = []
for text in tqdm(untokenized_texts):
    clean_text = ' '.join([word for word in text.split() if word_cnt[word] > 10])
    clean_texts.append(clean_text)

100%|████████████████████████████████████████████████████████████████████| 1600000/1600000 [00:05<00:00, 273359.74it/s]


In [9]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(clean_texts)
padded_sequences = pad_sequences(tokenizer.texts_to_sequences(clean_texts), padding = 'post')
max_len = padded_sequences.shape[1]
word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_LEN))
for word, i in tqdm(word_index.items()):
    embedding_matrix[i] = english_embedding_dict.get(word, np.zeros(EMBEDDING_LEN))

100%|████████████████████████████████████████████████████████████████████████| 29342/29342 [00:00<00:00, 326029.44it/s]


In [28]:
model = Sequential()
model.add(Embedding(
    len(word_index) + 1,
    EMBEDDING_LEN,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=max_len,
    trainable=False,
))
model.add(LSTM(200))
model.add(Dropout(0.1))
model.add(Dense(500))
model.add(Dropout(0.1))
model.add(Dense(250))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-2), metrics=["accuracy"])

In [29]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 200)           5868600   
_________________________________________________________________
lstm_6 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_17 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 500)               100500    
_________________________________________________________________
dropout_18 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 250)               125250    
_________________________________________________________________
dropout_19 (Dropout)         (None, 250)              

In [30]:
texts_train, texts_test, values_train, values_test = train_test_split(padded_sequences, emotions, test_size = 0.85)

In [31]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9

In [32]:
callback = LearningRateScheduler(scheduler)
model.fit(texts_train, values_train, batch_size=512, callbacks=[callback], epochs=100) # , validation_data=(texts_test, values_test))

Epoch 1/100
469/469 [==============================] - 15s 27ms/step - loss: 1.4391 - accuracy: 0.4951
Epoch 2/100
469/469 [==============================] - 13s 28ms/step - loss: 0.6774 - accuracy: 0.5299
Epoch 3/100
469/469 [==============================] - 14s 30ms/step - loss: 0.4650 - accuracy: 0.78873s - loss: 0.4678 - accu - ETA: 2s - loss: 0.4674 - accu - ETA: 2s - loss: 0.467 - ETA: 2s - loss:
Epoch 4/100
469/469 [==============================] - 14s 31ms/step - loss: 0.4150 - accuracy: 0.8138
Epoch 5/100
469/469 [==============================] - 16s 33ms/step - loss: 0.3833 - accuracy: 0.8305
Epoch 6/100
469/469 [==============================] - 16s 34ms/step - loss: 0.3584 - accuracy: 0.8427
Epoch 7/100
469/469 [==============================] - 16s 33ms/step - loss: 0.3321 - accuracy: 0.8587
Epoch 8/100
469/469 [==============================] - 17s 35ms/step - loss: 1.5979 - accuracy: 0.8601
Epoch 9/100
469/469 [==============================] - 16s 35ms/step - loss: 1

469/469 [==============================] - 16s 35ms/step - loss: 0.2184 - accuracy: 0.9243
Epoch 76/100
469/469 [==============================] - 16s 35ms/step - loss: 0.2189 - accuracy: 0.9239
Epoch 77/100
469/469 [==============================] - 16s 35ms/step - loss: 0.2184 - accuracy: 0.9244
Epoch 78/100
469/469 [==============================] - 17s 36ms/step - loss: 0.2184 - accuracy: 0.9243
Epoch 79/100
469/469 [==============================] - 17s 35ms/step - loss: 0.2158 - accuracy: 0.9249
Epoch 80/100
469/469 [==============================] - 17s 35ms/step - loss: 0.2167 - accuracy: 0.9248
Epoch 81/100
469/469 [==============================] - 16s 35ms/step - loss: 0.2155 - accuracy: 0.9256
Epoch 82/100
469/469 [==============================] - 16s 35ms/step - loss: 0.2175 - accuracy: 0.9247
Epoch 83/100
469/469 [==============================] - 17s 35ms/step - loss: 0.2166 - accuracy: 0.9243
Epoch 84/100
469/469 [==============================] - 16s 35ms/step - loss:

In [43]:
model.evaluate(texts_test, values_test, batch_size=512)

2657/2657 [==============================] - 29s 11ms/step - loss: 0.7685 - accuracy: 0.7522


[0.7684650421142578, 0.7521801590919495]

In [33]:
def predict_emotion(text):
    model_input = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=max_len, padding="post")
    model_output = sum(np.array(model.predict(model_input)[0]) * np.array([0, 1, 2, 3, 4]))
    return f"Sentence is positive for {(model_output / 4) * 100} %"

In [42]:
predict_emotion("I feel tired after working long hours")

'Sentence is positive for 7.814992964267731 %'

In [38]:
with open('tokenizer_1_6B_set.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

model.save('model_1_6B_set_LSTM_1_hidden_layer')

INFO:tensorflow:Assets written to: model_1_6B_set_LSTM_1_hidden_layer\assets


INFO:tensorflow:Assets written to: model_1_6B_set_LSTM_1_hidden_layer\assets
